In [2]:
import json

with open('dev.json', 'r') as f:
    dataset = json.load(f)

In [3]:
dataset

{'sst2': [{'idx': 0,
   'sentence': "it 's an uneven treat that bores fun at the democratic exercise while also examining its significance for those who take part .",
   'label': 1},
  {'idx': 1,
   'sentence': 'it captures a wonderful kind of laziness to waste the talents of robert forster , anne meara , eugene levy , and reginald veljohnson all in the same magic .',
   'label': 0},
  {'idx': 2,
   'sentence': "in exactly 89 minutes , most of which passed as slowly as if i 'd been sitting naked on an igloo , formula 51 broke from quirky to jerky to perfect swift .",
   'label': 0},
  {'idx': 3,
   'sentence': "even horror fans will most likely not find what they 're seeking with complaint every day ; the travelogue needs both thrills and humor .",
   'label': 0},
  {'idx': 4,
   'sentence': 'there is nothing outstanding about this film , but it is inoffensive enough and will only be appreciated most by sailors and folks who know their way around a submarine .',
   'label': 1},
  {'idx

In [21]:
from google.colab import userdata
my_key = userdata.get('key')

In [22]:
import os
os.environ["OPENAI_API_KEY"] = my_key

In [23]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [33]:
def get_gpt35_prediction(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content.strip()

In [35]:
def create_prompt(task, item):
    if task == 'sst2':
        return f"Classify the sentiment of the following sentence: \"{item['sentence']}\". Options: 0 (negative), 1 (positive)."
    elif task == 'qqp':
        return f"Are the following two questions asking the same thing? \"{item['question1']}\" and \"{item['question2']}\". Options: 0 (not equivalent), 1 (equivalent)."
    elif task == 'mnli' or task == 'mnli-mm':
        return f"Premise: \"{item['premise']}\" Hypothesis: \"{item['hypothesis']}\". Does the premise entail the hypothesis? Options: 0 (entailment), 1 (neutral), 2 (contradiction)."
    elif task == 'qnli':
        return f"Question: \"{item['question']}\" Sentence: \"{item['sentence']}\". Does the sentence answer the question? Options: 0 (entailment), 1 (not entailment)."
    elif task == 'rte':
        return f"Sentence 1: \"{item['sentence1']}\" Sentence 2: \"{item['sentence2']}\". Does the first sentence entail the second sentence? Options: 0 (entailment), 1 (not entailment)."
    else:
        raise ValueError(f"Unknown task: {task}")

predictions = {}
for task, items in dataset.items():
    task_predictions = []
    for item in items:
        prompt = create_prompt(task, item)
        prediction = get_gpt35_prediction(prompt)
        # Convert prediction to expected format
        if task in ['sst2', 'qqp', 'qnli', 'rte']:
            prediction = 1 if '1' in prediction else 0
        elif task in ['mnli', 'mnli-mm']:
            if 'entailment' in prediction.lower():
                prediction = 0
            elif 'neutral' in prediction.lower():
                prediction = 1
            else:
                prediction = 2
        task_predictions.append({'idx': item['idx'], 'prediction': prediction})
    predictions[task] = task_predictions

# Save predictions to a file
with open('predictions.json', 'w') as f:
    json.dump(predictions, f, indent=4)